In [1]:
!pip install --upgrade pip
!pip install transformers datasets accelerate peft safetensors huggingface_hub sentencepiece streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 12.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 104.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 131.8 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 126.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 86.6 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 126.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 51.3 MB/s  0:00:06m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from huggingface_hub import login

HF_TOKEN = os.getenv("HF_TOKEN")
if not HF_TOKEN:
    try:
        from kaggle_secrets import UserSecretsClient
        HF_TOKEN = UserSecretsClient().get_secret("HF_TOKEN")
        os.environ["HF_TOKEN"] = HF_TOKEN
        os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN
    except Exception as e:
        raise ValueError("❌ Add your token as a Kaggle Secret called HF_TOKEN") from e

login(token=HF_TOKEN)
print("✅ Logged into Hugging Face Hub!")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


✅ Logged into Hugging Face Hub!


In [3]:
from datasets import Dataset
data = [
    {"instruction": "Make this polite: 'Send the report now.'", "input": "", "output": "Could you please send the report when you get a moment? Thanks!"},
    {"instruction": "Make this friendlier: 'Submit the form.'", "input": "", "output": "When you have a minute, please submit the form — much appreciated!"},
    {"instruction": "Simplify this: 'Due to unforeseen circumstances, we must reschedule.'", "input": "", "output": "We have to reschedule because of unexpected issues."},
]
ds = Dataset.from_list(data)
ds.save_to_disk("/kaggle/working/tiny_humanizer_dataset")
print("✅ Dataset saved!")

Saving the dataset (0/1 shards):   0%|          | 0/3 [00:00<?, ? examples/s]

✅ Dataset saved!


In [7]:
from transformers import AutoTokenizer
from datasets import load_from_disk

base_model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

ds = load_from_disk("/kaggle/working/tiny_humanizer_dataset")
max_length = 128  # Reduced for consistency

def make_example(example):
    instr = example["instruction"].strip()
    inp = example.get("input", "").strip()
    out = example["output"].strip()

    prompt = f"### Instruction:\n{instr}\n\n### Input:\n{inp}\n\n### Response:\n"
    full = prompt + out

    # Tokenize with padding to max_length
    tokenized = tokenizer(
        full, 
        truncation=True, 
        max_length=max_length, 
        padding="max_length",
        add_special_tokens=True,
        return_tensors=None  # Return lists, not tensors
    )
    
    # Create labels - make a copy of input_ids list
    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]
    labels = list(input_ids)  # Create new list
    
    # Mask the prompt part (find where "Response:" ends)
    tokenized_prompt = tokenizer(prompt, add_special_tokens=True)
    prompt_len = len(tokenized_prompt["input_ids"])
    
    # Set prompt tokens to -100 (ignore in loss)
    for i in range(min(prompt_len, len(labels))):
        labels[i] = -100
    
    # Also mask padding tokens
    for i, token_id in enumerate(input_ids):
        if token_id == tokenizer.pad_token_id:
            labels[i] = -100

    return {
        "input_ids": input_ids, 
        "attention_mask": attention_mask,
        "labels": labels
    }

ds = ds.map(make_example, remove_columns=ds.column_names)
print("✅ Tokenized examples ready.")
print(f"Dataset columns: {ds.column_names}")
print(f"First example keys: {list(ds[0].keys())}")

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

✅ Tokenized examples ready.
Dataset columns: ['input_ids', 'attention_mask', 'labels']
First example keys: ['input_ids', 'attention_mask', 'labels']


In [8]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, TaskType
import torch

base_model = AutoModelForCausalLM.from_pretrained(base_model_name)
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_attn"],  # Simplified for GPT-2
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(base_model, lora_config)
if hasattr(model, "config"):
    model.config.use_cache = False

# Data collator - no need for additional padding since we already padded
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False,
    pad_to_multiple_of=None
)

# Set format for PyTorch
ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

training_args = TrainingArguments(
    output_dir="/kaggle/working/humanizer-lora",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=False,
    logging_steps=1,
    save_total_limit=2,
    remove_unused_columns=False,
    report_to="none",
    dataloader_drop_last=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds,
    data_collator=data_collator
)

print("🚀 Starting training...")
trainer.train()
print("✅ Training complete!")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


🚀 Starting training...


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
1,2.082400
2,5.090800
3,2.305800
4,4.086700
5,2.265200
6,4.602600


✅ Training complete!


In [9]:
model.save_pretrained("/kaggle/working/humanizer-lora")
tokenizer.save_pretrained("/kaggle/working/humanizer-lora")
print("✅ LoRA adapter saved successfully!")

✅ LoRA adapter saved successfully!


In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/humanizer-lora")
base = AutoModelForCausalLM.from_pretrained(base_model_name)
inferred = PeftModel.from_pretrained(base, "/kaggle/working/humanizer-lora")
inferred.eval()

prompt = "### Instruction:\nMake this more polite: 'Send it now.'\n\n### Response:\n"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = inferred.generate(**inputs, max_new_tokens=80, do_sample=True, temperature=0.8)
print(tokenizer.decode(outputs[0], skip_special_tokens=True).split('### Response:')[-1].strip())

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


To all 'send it more quickly.' requests:

Make this polite: 'Send it more efficiently.'
